In [4]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import utils
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format', lambda x: '%.10f' % x)


In [5]:
# load data
df = utils.load_suicide_data()

In [6]:
df.head()

,country,year,sex,age,suicides_no,population,suicides_per_100k_pop,gdp_for_year,gdp_per_capita,generation,country_code
0,Albania,1987,male,15-24 years,21,312900,6.7100000000,"2,156,624,900",796,Generation X,ALB
1,Albania,1987,male,35-54 years,16,308000,5.1900000000,"2,156,624,900",796,Silent,ALB
2,Albania,1987,female,15-24 years,14,289700,4.8300000000,"2,156,624,900",796,Generation X,ALB
3,Albania,1987,male,75+ years,1,21800,4.5900000000,"2,156,624,900",796,G.I. Generation,ALB
4,Albania,1987,male,25-34 years,9,274300,3.2800000000,"2,156,624,900",796,Boomers,ALB
